Recreacion del codigo para ejecutar el algoritmo txmeans, puediendolo depurar de una manera mas eficiente

In [1]:
import sys
sys.path.insert(0, r'C:\Users\adria\Desktop\TFG\TXMeans\code')

In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from generators.datamanager import *
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import normalize
import datetime
from tabulate import tabulate
results = []

In [3]:
#parametros a probar
vector_size = 100
window = 10
epochs = 50


In [4]:
datapath = "../../dataset/mushrooms.csv"

df = pd.read_csv(datapath)
missing_symbol='?'
index_mode = dict()

for k, index in zip(df.columns, range(len(df.columns))):
    df[k] = df[k].replace(missing_symbol, np.nan)
    mode_value = df[k].mode()[0]
    df[k] = df[k].fillna(mode_value)
    index_mode[index] = mode_value

columns_to_use = df.columns[df.columns != 'class']


In [5]:
df['processed_text'] = df[columns_to_use].apply(lambda row: ' '.join(row.astype(str)), axis=1)


In [6]:
df['processed_text'] = df['processed_text'].apply(lambda x: x.split())

In [7]:
model = Word2Vec(sentences=df['processed_text'], vector_size=vector_size, window=window, min_count=1, workers=4, sg=1, epochs=epochs)

In [8]:

def get_average_vector(words, model):
    vectors = []
    for word in words:
        if word in model.wv:
            vectors.append(normalize(model.wv[word].reshape(1, -1))[0])
    if vectors:
        return normalize(np.mean(vectors, axis=0).reshape(1, -1))[0]
    else:
        return np.zeros(model.vector_size)
    



X = np.array([get_average_vector(text, model) for text in df['processed_text']])
print(X.shape)


(8124, 100)


In [9]:
n_clusters = 2
start_time = datetime.datetime.now()
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
end_time = datetime.datetime.now()
running_time = end_time - start_time


y_kmeans = kmeans.predict(X)

label_map = {'p': 0, 'e': 1}
real_labels_numeric = df['class'].map(label_map)

nmi = normalized_mutual_info_score(real_labels_numeric, y_kmeans)

results.append({
    "vector_size": vector_size, 
    "window": window, 
    "epochs": epochs, 
    "NMI": nmi, 
    "running_time": running_time
})
print(tabulate(results, headers="keys", tablefmt="fancy_grid"))

╒═══════════════╤══════════╤══════════╤══════════╤════════════════╕
│   vector_size │   window │   epochs │      NMI │ running_time   │
╞═══════════════╪══════════╪══════════╪══════════╪════════════════╡
│           100 │       10 │       50 │ 0.238949 │ 0:00:00.204200 │
╘═══════════════╧══════════╧══════════╧══════════╧════════════════╛
